## Example

In [1]:
import pandas as pd
import numpy as np

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

print("I_0:", I_0)
print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
for i in range(N):
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

print("C:", C)
print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

######################## PARAMTERS ENDING HERE ##################################

N: 10 T: 6
I_0: [800. 600. 425. 350. 400. 524. 453. 218. 673. 200.]
D: [[138.  55. 172. 194.  94. 185.]
 [190. 101.  68. 185.  13. 136.]
 [ 79. 179.  21.  49. 199. 200.]
 [142. 103.  78. 131. 146. 155.]
 [ 35.  62.  83.  90. 197.  49.]
 [ 91.  95. 107. 127. 116. 183.]
 [105. 164.  19. 116. 119. 175.]
 [ 37. 155.  10.  77. 168.  32.]
 [108. 185. 188. 176.  81. 172.]
 [ 46. 178. 162. 200. 154. 199.]]
I: [[  0.   0.   0.   0.   0.   0.]
 [ 48.   0.   0.   0.   0.   0.]
 [  0.  20.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 18.  23.   0.   0.   0.   0.]
 [ 28.  45.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [109.  34.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
C: {'H': array([100.,  40., 180., 180.,  40., 180., 140., 100., 180., 140.]), 'P': array([5000., 2000., 9000., 9000., 2000., 9000., 7000., 5000., 9000.,
       7000.]), 'V': array([[44., 18.,  0.],
       [89., 45.,  0.],
       [86., 38.,  0.],
       [91., 46., 

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Set parameter Username
Set parameter LicenseID to value 2633994
Academic license - for non-commercial use only - expires 2026-03-09
Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0

Optimize a model with 398 rows, 264 columns and 838 nonzeros
Model fingerprint: 0xbe75289a
Variable types: 240 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 1.967757e+07
Presolve removed 365 rows and 122 columns
Presolve time: 0.02s
Presolved: 33 rows, 142 columns, 300 nonzeros
Found heuristic solution: objective 1.697771e+07
Variable types: 128 continuous

In [3]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 16890367.020408165

Order quantities (x_ijt):
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[4,3,2] = 97.0
x[5,3,2] = 67.0
x[8,3,2] = 168.0
x[10,3,2] = 36.44897959183673
x[1,3,3] = 38.0
x[2,3,3] = 45.0
x[3,3,3] = 200.0
x[4,3,3] = 155.0
x[5,3,3] = 49.0
x[6,3,3] = 154.0
x[7,3,3] = 172.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 117.55102040816327
x[10,3,3] = 199.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3] = 200.0
v[1,4] = 241.0
v[2,4] = 104.0
v[3,4] = 199.0
v[4,4] = 146.0
v[5,4] = 197.0
v[6,4] = 145.0


In [ ]:
#original problem, only adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = list()

for j in S_T:
    if j == 0:
        J_in_inventory.append(1)
    elif j == 1:
        J_in_inventory.append(2)
    else:
        J_in_inventory.append(3)

J_in_inventory = np.array(J_in_inventory)

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

## Solve Linear relaxation

In [2]:
#linear relaxation
#also adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB


def LR(N,T,J,C,I,I_0,D,V,V_C):
    # Provided parameters (already read from the Excel file)
    # N: number of products, T: number of time periods, J: number of shipping methods
    # D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

    # Create the Gurobi model
    model = gp.Model("InventoryManagement")

    # Set error parameter
    model.setParam('MIPGap', 0.0)

    # Define sets
    S_I = range(N)  # Products i in {0,  ..., N-1}
    S_T = range(T)  # Time periods t in {0, ..., T-1}
    S_J = range(J)  # Shipping methods j in {0, ..., J-1}

    # Variables
    x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
    v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
    y = model.addVars(S_J, S_T, vtype=GRB.CONTINUOUS, name="y")  # Relaxed: Binary for shipping method y_jt
    z = model.addVars(S_T, vtype=GRB.CONTINUOUS, name="z")  # Relaxed: Number of containers z_t

    # Objective function (1)
    # Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
    holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
    purchasing_and_shipping_cost = gp.quicksum(
        (C["P"][i] + C["V"][i, j]) * x[i, j, t]
        for i in S_I for j in S_J for t in S_T
    ) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
    container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

    model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

    # Constraints
    # Inventory balance (2)
    J_in_inventory = list()

    for j in S_T:
        if j == 0:
            J_in_inventory.append(1)
        elif j == 1:
            J_in_inventory.append(2)
        else:
            J_in_inventory.append(3)

    for i in S_I:
        for t in S_T:
            # Compute the in-transit quantity arriving at time t
            in_inventory = 0
            for j in range(J_in_inventory[t]):
                in_inventory += x[i, j, t - T_lead[j] + 1]
            # Add the constraint for inventory balance
            if t == 0:
                model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            else:
                model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
                model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

    # Relate order quantity and shipping method (4)
    M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
    for j in S_J:
        for t in S_T:
            model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

    # Container constraint (5)
    for t in S_T:
        model.addConstr(
            gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
            name=f"Container_{t}"
        )

    # Non-negativity and binary constraints (6)
    for i in S_I:
        for j in S_J:
            for t in S_T:
                model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
    for i in S_I:
        for t in S_T:
            model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
    for j in S_J:
        for t in S_T:
            model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
            model.addConstr(y[j, t] <= 1, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
    for t in S_T:
        model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

    # Optimize the model
    model.optimize()
    return(model.objVal, model.Runtime)

## Solve Naive

In [3]:
#linear relaxation
#use express only, always order the number we need in next month
import gurobipy as gp
from gurobipy import GRB
import time

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

def naive(N,T,J,C,I,I_0,D,V,V_C):
    # Create the Gurobi model

    start_time = time.time() # Start the timer
    
    model = gp.Model("InventoryManagement")

    # Set error parameter
    model.setParam('MIPGap', 0.0)

    # Define sets
    S_I = range(N)  # Products i in {0,  ..., N-1}
    S_T = range(T)  # Time periods t in {0, ..., T-1}
    S_J = range(J)  # Shipping methods j in {0, ..., J-1}

    # Variables
    x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
    v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
    y = model.addVars(S_J, S_T, vtype=GRB.CONTINUOUS, name="y")  # Relaxed: Binary for shipping method y_jt
    z = model.addVars(S_T, vtype=GRB.CONTINUOUS, name="z")  # Relaxed: Number of containers z_t

    # Objective function (1)
    # Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
    holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
    purchasing_and_shipping_cost = gp.quicksum(
        (C["P"][i] + C["V"][i, j]) * x[i, j, t]
        for i in S_I for j in S_J for t in S_T
    ) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
    container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

    model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

    # Constraints

    #new constraints: each time we order for exactly next month's demand, use express only, ignore product in transit and initial inventory
    for i in S_I:
        for t in S_T:
            if t == len(S_T) - 1:
                model.addConstr(x[i, 0, t] == 0, name=f"last month no order")
            else:
                model.addConstr(x[i, 0, t] == D[i,t+1], name=f"order for next month demand")

            model.addConstr(x[i, 1, t] == 0, name=f"No air freight")
            model.addConstr(x[i, 2, t] == 0, name=f"No ocean freight")


    # Inventory balance (2)
    J_in_inventory = list()

    for j in S_T:
        if j == 0:
            J_in_inventory.append(1)
        elif j == 1:
            J_in_inventory.append(2)
        else:
            J_in_inventory.append(3)

    for i in S_I:
        for t in S_T:
            # Compute the in-transit quantity arriving at time t
            in_inventory = 0
            for j in range(J_in_inventory[t]):
                in_inventory += x[i, j, t - T_lead[j] + 1]
            # Add the constraint for inventory balance
            if t == 0:
                model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            else:
                model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
                model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

    # Relate order quantity and shipping method (4)
    M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
    for j in S_J:
        for t in S_T:
            model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

    # Container constraint (5)
    for t in S_T:
        model.addConstr(
            gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
            name=f"Container_{t}"
        )

    # Non-negativity and binary constraints (6)
    for i in S_I:
        for j in S_J:
            for t in S_T:
                model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
    for i in S_I:
        for t in S_T:
            model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
    for j in S_J:
        for t in S_T:
            model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
            model.addConstr(y[j, t] <= 1, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
    for t in S_T:
        model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

    # Optimize the model
    model.optimize()

    # return the objective value and computation time
    computation_time = time.time() - start_time # End the timer
    return(model.objVal, computation_time)

In [21]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 46815670.65108606

Order quantities (x_ijt):
x[1,1,1] = 55.0
x[2,1,1] = 101.0
x[3,1,1] = 179.0
x[4,1,1] = 103.0
x[5,1,1] = 62.0
x[6,1,1] = 95.0
x[7,1,1] = 164.0
x[8,1,1] = 155.0
x[9,1,1] = 185.0
x[10,1,1] = 178.0
x[1,1,2] = 172.0
x[2,1,2] = 68.0
x[3,1,2] = 21.0
x[4,1,2] = 78.0
x[5,1,2] = 83.0
x[6,1,2] = 107.0
x[7,1,2] = 19.0
x[8,1,2] = 10.0
x[9,1,2] = 188.0
x[10,1,2] = 162.0
x[1,1,3] = 194.0
x[2,1,3] = 185.0
x[3,1,3] = 49.0
x[4,1,3] = 131.0
x[5,1,3] = 90.0
x[6,1,3] = 127.0
x[7,1,3] = 116.0
x[8,1,3] = 77.0
x[9,1,3] = 176.0
x[10,1,3] = 200.0
x[1,1,4] = 94.0
x[2,1,4] = 13.0
x[3,1,4] = 199.0
x[4,1,4] = 146.0
x[5,1,4] = 197.0
x[6,1,4] = 116.0
x[7,1,4] = 119.0
x[8,1,4] = 168.0
x[9,1,4] = 81.0
x[10,1,4] = 154.0
x[1,1,5] = 185.0
x[2,1,5] = 136.0
x[3,1,5] = 200.0
x[4,1,5] = 155.0
x[5,1,5] = 49.0
x[6,1,5] = 183.0
x[7,1,5] = 175.0
x[8,1,5] = 32.0
x[9,1,5] = 172.0
x[10,1,5] = 199.0

Ending inventory (v_it):
v[1,1] = 717.0
v[2,1] = 559.0
v[3,1] = 525.0
v[4,1] = 464.0
v[5,1

## Sampling

In [4]:
import random

a = random.uniform(10, 20) # [10, 20]
a

14.054621083968913

| Scale | Number products/Periods | Container | Holding cost |
|--|---------------------------------|-----------|--------------|
| small |   10/6                              |    1375       |        1%      |
| medium |      100/20                           |      2750     |       2%       |
| large |     500/50           |       5500    |       4%       | 

In [5]:
scale_dict = {
    "small":{"N": 10, "T": 6, "C_C": 1375, "C_H": 0.01},
    "medium":{"N": 100, "T": 20, "C_C": 2750, "C_H": 0.02},
    "large":{"N": 500, "T": 50, "C_C": 5500, "C_H": 0.04}
}

#low -> small
#high -> large
 
scenerio_dict = {
    1: {"scale": "medium", "C_C": "medium", "C_H": "medium"},
    2: {"scale": "small", "C_C": "medium", "C_H": "medium"},
    3: {"scale": "large", "C_C": "medium", "C_H": "medium"},
    4: {"scale": "medium", "C_C": "small", "C_H": "medium"},
    5: {"scale": "medium", "C_C": "large", "C_H": "medium"},
    6: {"scale": "medium", "C_C": "medium", "C_H": "small"},
    7: {"scale": "medium", "C_C": "medium", "C_H": "large"}
}

scenerio_id_list = list() # should contain 210 rows, each scenerio with 30 instances, so the list should be [1, 1, 1, ..., 2, 2, 2,...]
for i in range(1, 8):
    for j in range(1, 31):
        scenerio_id_list.append(i)

instance_id_list = range(1, 211) # should contain 210 rows, one for a instance, so the list should be [1, 2, 3,...,210]

In [10]:
file_path = "OR113-2_midtermProject_data.xlsx"

LR_obj_list = list()
naive_obj_list = list()
naive_optimality_gap = list()
LR_computation_time = list()
naive_computation_time = list()

## 7 scenarios
random.seed(42)
np.random.seed(42)
for scenerio in range(1,8): #(1,8)
    #for each scenario, generate 30 instances
    for instance in range(1, 31): #(1, 31)
        
        scaleof = scenerio_dict[scenerio]
        
        N = scale_dict[scaleof["scale"]]["N"]
        T = scale_dict[scaleof["scale"]]["T"]
        #print("N:", N, "T:", T) #print product number, period number in this scenario
        

        I = np.zeros([N, T])
        D = np.zeros([N, T])
        I_0 = np.zeros([N])
        
        for i in range(N):
            for t in range(T):
                D[i, t] = random.uniform(0, 200)
    
            I_0[i] = random.uniform(D[i, 0], 400)
        
        #print("I_0:", I_0) #initial inventory of i, not smaller than D[i,0]
        #print("D:", D) #demand D[i,t]
        
        
        # Read the Excel file from the 'In-transit' sheet ##it's needed??
        #df_in_transit = pd.read_excel(file_path, sheet_name="In-transit") 

        for i in range(N):
            for t in range(T):
                if t == 0: # first period
                    I[i, 0] = random.choice([0, 1]) * random.uniform(0, 200) # correspond to I_i,1
                elif t == 1: # second period
                    I[i, 0] = random.choice([0, 1]) * random.uniform(0, 50) # correspond to I_i,2                       
                else:
                    I[i, t] = 0
        
        #print("I:", I) #in transit
        
        
        # Read the Excel file from the 'Shipping cost' sheet ##it's needed??
        #df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost") 
        #J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
        #df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")
        
        #all costs
        C = {
            "H": np.zeros([N]),
            "P": np.zeros([N]),
            "V": np.zeros([N, J]),
            "F": np.array([100, 80, 50]), #fixed
            "C": scale_dict[scaleof["C_C"]]["C_C"], ##找到scenario對應的container cost (originally 2750)
        }

        
        V = np.zeros([N])#vi: volume of product i
        V_C = 30
        for i in range(N):
            C["P"][i] = random.uniform(1000, 10000)
            C["H"][i] = C["P"][i] * scale_dict[scaleof["C_H"]]["C_H"] ##找到scenario對應的holding cost比例 
            
            V[i] = random.uniform(0, 1)

            #variable cost of two shipping method 
            C["V"][i, 0] = 0
            C["V"][i, 1] = random.uniform(40, 100)
            C["V"][i, 2] = random.uniform(0.4, 0.6) # No variable shipping cost for ocrean freight
        
        #print("C:", C)
        #print("V:", V)
        T_lead = np.array([1, 2, 3]) # T_j, lead time of 3 shipping methods
    

        LR_obj, LR_time = LR(N,T,J,C,I,I_0,D,V,V_C)
        LR_obj_list.append(LR_obj)
        LR_computation_time.append(LR_time)

        naive_obj, naive_time = naive(N,T,J,C,I,I_0,D,V,V_C)
        naive_obj_list.append(naive_obj)
        naive_computation_time.append(naive_time)

        naive_optimality_gap.append((naive_obj - LR_obj) / LR_obj)

Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0

Optimize a model with 12120 rows, 8080 columns and 27720 nonzeros
Model fingerprint: 0xb3493fff
Coefficient statistics:
  Matrix range     [1e-02, 2e+05]
  Objective range  [2e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 4e+02]
Presolve removed 10327 rows and 887 columns
Presolve time: 0.02s
Presolved: 1793 rows, 7193 columns, 14232 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2322629e+07   2.194448e+04   0.000000e+00      0s
    3207    9.1043732e+08   0.000000e+00   0.000000e+00      0s

Solved in 3207 iterations and 0.05 seconds (0.02 work units)
Optimal objective  9.104373202e+08
Set parameter

In [11]:
print(LR_obj_list)

[910437320.1712431, 958556275.0807928, 973547649.9725811, 992092652.3887901, 872325306.8450257, 1033062887.1174656, 906883480.0417651, 924327709.8927268, 973968229.2653648, 970524695.1562628, 871660390.8564374, 916367528.3492403, 898879502.5568131, 1019228780.949725, 936327174.564618, 983353764.3769771, 1011011825.0223538, 953416583.8697841, 945164690.9590337, 964540793.2627566, 1092192927.0359178, 1053415671.4640568, 974269268.031586, 1011625062.1593882, 1012044761.0439962, 984850183.6360974, 946503741.0634443, 1066338330.4161708, 975532301.7352574, 1014033222.4577637, 21087539.547314797, 20448695.729421042, 18571123.138002872, 24235415.470270127, 15316146.302293217, 13748736.512761576, 17874061.470721982, 17470061.33925307, 17296677.20562213, 19482665.773838963, 24361075.39457067, 18942808.77274743, 21596542.223255727, 23222251.94288941, 22483060.490883075, 18724069.922619868, 20371979.131359905, 16265837.919985844, 20947644.77466151, 6272074.464099809, 20654119.46383206, 15035630.57

In [8]:
print(naive_obj_list)

[1075783223.9687104, 1154794690.7412384, 1163020222.2399118]


In [9]:
print(naive_optimality_gap)

[0.15529676765267383, 0.1773867973377008, 0.167210204651706]


In [12]:
print(len(naive_optimality_gap))

210


In [15]:
data = {"LR":LR_obj_list,"naive":naive_obj_list,"gap":naive_optimality_gap}
df = pd.DataFrame(data)
df.head(3)

,LR,naive,gap
0,9.104373e+08,1.023637e+09,0.124336
1,9.585563e+08,1.094744e+09,0.142076
2,9.735476e+08,1.104219e+09,0.134222


In [17]:
df.to_csv("result.csv",index=False)

## Get both solution for all instances

## Save final result

In [ ]:
# The CSV or XLSX file should contain detailed experiment results in the fol-
# lowing format. It should contain 210 rows, one for each instance. It should
# contain the following columns: (1) scenario ID, (2) instance ID, (3)–(5) ob-
# jective values reported by the LP relaxation, proposed algorithm, and very
# naive heuristics, (6)–(8) computation times required to run the three solution
# approaches, and (9)–(10) two optimality gaps in percentages


save_data = { "scenario": scenerio_id_list, "instance": instance_id_list,
             "LR_obj": LR_obj_list, "heuristic_obj": naive_obj_list, "naive_obj": optimality_gap,
             "LR_time": LR_computation_time, "heuristic_time": [], "naive_time": naive_computation_time,
             "heuristic_gap": [], "naive_gap": naive_optimality_gap}

save_df = pd.DataFrame(save_data)

# Save to csv without header
save_df.to_csv("experiment_results.csv", index=False, header=False)